<a href="https://colab.research.google.com/github/RobertGodin/CodePython/blob/master/ExempleReglageFinBertTweetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Exemple de réglage fin de BERT pour analyse de sentiment de TWEET
# Installation des bibliothèques
%pip install numpy
%pip install pandas
%pip install transformers
%pip install datasets
%pip install evaluate
%pip install torch
%pip install huggingface_hub
%pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [2]:
# Chargement des données de tweet
nom_dataset = "mteb/tweet_sentiment_extraction"
import pandas as pd
from datasets import load_dataset
dataset_tweet = load_dataset(nom_dataset)
print(dataset_tweet)
df_ent = pd.DataFrame(dataset_tweet["train"])
df_ent.head(3)

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 27481
    })
    test: Dataset({
        features: ['id', 'text', 'label', 'label_text'],
        num_rows: 3534
    })
})


,id,text,label,label_text
0,cb774db0d1,"I`d have responded, if I were going",1,neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,0,negative
2,088c60f138,my boss is bullying me...,0,negative


In [4]:
# Chargement du modèle et tokenizer Bert de Google
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
nom_modele = 'google-bert/bert-base-uncased'
modele = BertForSequenceClassification.from_pretrained(nom_modele, num_labels=3)
tokenizer = BertTokenizer.from_pretrained(nom_modele)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Vectorisation des textes pour l'entrainement
def tokenize_text(data):
  return tokenizer(data["text"], padding = "max_length",truncation=True)
dataset_vectorise = dataset_tweet.map(tokenize_text, batched=True)
dataset_ent = dataset_vectorise["train"].shuffle(seed=42).select(range(500))  #petite collection de 100
dataset_test = dataset_vectorise["test"].shuffle(seed=42).select(range(500))
df_ent=pd.DataFrame(dataset_ent)
df_ent.head(3)

Map:   0%|          | 0/27481 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

,id,text,label,label_text,input_ids,token_type_ids,attention_mask
0,fd8e3ede29,what happen to the maids of yours? Quit once ...,1,neutral,"[101, 2054, 4148, 2000, 1996, 29229, 1997, 673...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,9206b5221d,Listening to the commentary track on Holiday I...,2,positive,"[101, 5962, 2000, 1996, 8570, 2650, 2006, 6209...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,d6b51b539f,"chatting with some of my old classmates, helly...",1,neutral,"[101, 22331, 2007, 2070, 1997, 2026, 2214, 198...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
# Fonction pour prédire le sentiment avec le modèle
import torch
def analyse_sentiment(texte):
    texte_entree = tokenizer(texte, return_tensors="pt", padding=True, truncation=True, max_length=512)
    predictions = modele(**texte_entree)
    prediction_classe = torch.argmax(predictions.logits, dim=1)
    classe_etiquettes = {0: "negative", 1: "neutral", 2: "positive"}
    sentiment = classe_etiquettes[prediction_classe.item()]
    return sentiment
print("Analyse de sentiment pour 'This is a very nice tweet' est: ", analyse_sentiment('This is a very nice tweet'))

Analyse de sentiment pour 'This is a very nice tweet' est:  neutral


In [ ]:
from transformers import Trainer, TrainingArguments
arguments_training = TrainingArguments(
    output_dir='dossier_bert_ft_tweet', # Dossier pour les résultats
    evaluation_strategy="epoch",

    num_train_epochs=10, # Défaut est 3
    learning_rate=2e-5, # Taux d'apprentissage
    per_device_train_batch_size=4, # Taille des mini-lots
    per_device_eval_batch_size=4,
    weight_decay=0.01, # Pour régularisation L2
)
import evaluate
import numpy as np
exactitude = evaluate.load("accuracy")
def calcul_metriques(p):
    logits, etiquettes = p
    predictions = np.argmax(logits, axis=1)
    return exactitude.compute(predictions=predictions, references=etiquettes)

entraineur = Trainer(
    model=modele,
    args=arguments_training,
    train_dataset=dataset_ent,
    eval_dataset=dataset_test,
    compute_metrics=calcul_metriques
)
entraineur.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Sauver le résultat de l'entrainement
trainer.save_model("bert_ft_tweet")
tokenizer.save_pretrained("bert_ft_tweet")
# Recharger le modèle entrainé
model = BertForSequenceClassification.from_pretrained("bert_ft_tweet")
tokenizer = BertTokenizer.from_pretrained("bert_ft_tweet")

print("Analyse de sentiment pour 'This is a very nice tweet' est: ", analyse_sentiment('This is a very nice tweet'))



The sentiment for 'how on earth can I analyse this?' is:  negative
